In [14]:
import pandas as pd
import geopandas as gpd
import requests

# 1. โหลดข้อมูลแปลงเกษตร
farms = gpd.read_file(r"E:\PythonFor_Flood\drive-download-20260119T075939Z-3-001\พระนครศรีอยุธยา.shp")
print(f"จำนวนแปลงเกษตรตั้งต้น: {len(farms)} แปลง")

จำนวนแปลงเกษตรตั้งต้น: 355874 แปลง


In [ ]:
from shapely.validation import make_valid # นำเข้าฟังก์ชัน fix geometry

# 2.---Fix Geometry (เหมือน Fix Geometry ใน QGIS) ---
print("กำลังตรวจสอบและซ่อมแซม Geometry (Fix Geometry)...")

# ตรวจสอบว่ามีแปลงที่ Invalid หรือไม่
invalid_count = (~farms.is_valid).sum()
print(f"พบ Geometry ที่มีปัญหา: {invalid_count:,} แปลง")

if invalid_count > 0:
    # ใช้ .make_valid() เพื่อซ่อมแซมรูปทรง)
    farms['geometry'] = farms['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
    print("ซ่อมแซม Geometry เรียบร้อยแล้ว")
else:
    print("Geometry ทั้งหมดสมบูรณ์")
# --------------------------------------------------------

กำลังตรวจสอบและซ่อมแซม Geometry (Fix Geometry)...
พบ Geometry ที่มีปัญหา: 0 แปลง
Geometry ทั้งหมดสมบูรณ์ดี


In [ ]:
# 3. ฟังก์ชันดึง API ด้วย BBox
def get_bbox(gdf):
    minx, miny, maxx, maxy = gdf.total_bounds
    return f"{minx},{miny},{maxx},{maxy}"

#API 
api_url = f"https://api-gateway.gistda.or.th/api/2.0/resources/features/flood-freq?api_key=YOUR_API_KEY&bbox={get_bbox(farms)}&collectionCreatedBy=66b07c8034c2963abcdb9bb8&limit=10000&offset=0"
response = requests.get(api_url)
flood_gdf = gpd.GeoDataFrame.from_features(response.json()['features'], crs="EPSG:4326")

print({get_bbox(farms)})
print("--- ผลลัพธ์ข้อมูลน้ำท่วมจาก GISTDA API ---")
print(flood_gdf[['LabelTH', 'freq', 'geometry']].head(3))
print(flood_gdf['freq'].value_counts().sort_index())

{'100.214415422,14.106310382171,100.829815,14.676963474260967'}
--- ผลลัพธ์ข้อมูลน้ำท่วมจาก GISTDA API ---
         LabelTH  freq                                           geometry
0  น้ำท่วมซ้ำซาก     1  MULTIPOLYGON (((100.50959 14.11327, 100.50958 ...
1  น้ำท่วมซ้ำซาก     1  MULTIPOLYGON (((100.51842 14.11443, 100.51839 ...
2  น้ำท่วมซ้ำซาก     1  MULTIPOLYGON (((100.51055 14.11468, 100.51033 ...
freq
1    3750
2    5612
3     629
4       9
Name: count, dtype: int64


In [ ]:
# 4. แปลงพิกัดเป็น UTM เพื่อความแม่นยำ
farms_utm = farms.to_crs(epsg=32647)
flood_utm = flood_gdf.to_crs(epsg=32647)
#เช็กว่า CRS ตรงกันจริงไหมก่อน Join
print(f"Farms CRS: {farms_utm.crs}")
print(f"Flood CRS: {flood_utm.crs}")



Farms CRS: EPSG:32647
Flood CRS: EPSG:32647


In [ ]:

# 5.sjoin แบบ left: เก็บทุกแปลงไว้ (Eliminate missing data จากการ Intersect)
joined_data = gpd.sjoin(farms_utm, flood_utm[['freq', 'geometry']], how='left', predicate='intersects')

#ลองเช็กจำนวนแปลงที่ "ท่วม" เทียบกับ "ทั้งหมด"
total = len(joined_data)
flooded = joined_data['freq'].notna().sum()



print("--- ผลลัพธ์หลังการ Join (ยังไม่ Clean) ---")
print(joined_data[['act','type_name', 'detail_nam','plant_date','year_act','p_name', 'a_name','t_name','moo','freq']].head(5))
print(f"จากทั้งหมด {total:,} แปลง | พบพื้นที่น้ำท่วม {flooded:,} แปลง")

--- ผลลัพธ์หลังการ Join (ยังไม่ Clean) ---
         act type_name detail_nam plant_date year_act           p_name  \
0  177385400      ข้าว   ข้าวเจ้า   20220510     2565  พระนครศรีอยุธยา   
1  167491913      ข้าว   ข้าวเจ้า   20220501     2565  พระนครศรีอยุธยา   
2  163059714      ข้าว   ข้าวเจ้า   20220508     2565  พระนครศรีอยุธยา   
3  172986624      ข้าว   ข้าวเจ้า   20220727     2565  พระนครศรีอยุธยา   
4  167737418      ข้าว   ข้าวเจ้า   20220427     2565  พระนครศรีอยุธยา   

     a_name  t_name moo  freq  
0    บางไทร  บางพลี   5   NaN  
1  บางปะหัน  ตาลเอน   2   NaN  
2  บางปะหัน  ตาลเอน   2   NaN  
3   ท่าเรือ    จำปา   8   NaN  
4  บางปะหัน  ตาลเอน   1   NaN  
จากทั้งหมด 366,618 แปลง | พบพื้นที่น้ำท่วม 49,788 แปลง


In [ ]:
# 6. เรียงลำดับความสำคัญ: 
# - เลขประจำกิจกรรม (act) 
# - ปีล่าสุด (year_act) -> เรียงจากมากไปน้อย (False)
# - ความเสี่ยงสูงสุด (freq) -> เรียงจากมากไปน้อย (False)
joined_data_sorted = joined_data.sort_values(
    by=['act', 'year_act', 'freq'], 
    ascending=[True, False, False]
)

# 7. ลบตัวซ้ำ (Deduplicate)
# จะเก็บเฉพาะ "แถวแรก" ของแต่ละ 'act' (ซึ่งก็คือแถวที่มีปีใหม่สุดและ freq สูงสุด)
cleaned_farms = joined_data_sorted.drop_duplicates(subset=['act'], keep='first').copy()

print(f"--- สรุปการจัดการข้อมูลซ้ำ ---")
print(f"จำนวนข้อมูลที่ Join มาทั้งหมด: {len(joined_data):,} แถว")
print(f"จำนวนแปลงเกษตรที่ Clean แล้ว: {len(cleaned_farms):,} แปลง")
print(f"จำนวนแปลงที่ลดลงไป (ตัวซ้ำ): {len(joined_data) - len(cleaned_farms):,} แถว")

# ตรวจสอบว่าจำนวนแปลงกลับมาใกล้เคียงค่าตั้งต้นหรือยัง (ควรกลับมาเท่ากับจํานวนขั้นตอนแรก)

--- สรุปการจัดการข้อมูลซ้ำ ---
จำนวนข้อมูลที่ Join มาทั้งหมด: 366,618 แถว
จำนวนแปลงเกษตรที่ Clean แล้ว: 355,874 แปลง
จำนวนแปลงที่ลดลงไป (ตัวซ้ำ): 10,744 แถว


In [ ]:
# 8. จัดกลุ่มพืช 5 กลุ่ม (Group Mapping)
def crop_classification(crop):
    if pd.isna(crop): return '5-อื่นๆ'
    crop = str(crop)
    if 'ข้าว' in crop: return '1-ข้าว'
    if 'พืชไร่' in crop: return '2-พืชไร่'
    if 'พืชสวน' in crop: return '3-พืชสวน'
    if 'พืชผัก' in crop: return '4-พืชผัก'
    return '5-อื่นๆ' # เพิ่มบรรทัดนี้กันเหนียวเผื่อข้อมูลไม่เข้าเงื่อนไขด้านบน
    

cleaned_farms['crop_group'] = cleaned_farms['type_name'].apply(crop_classification)

#  วิเคราะห์สถานะความเสี่ยง (ใช้ปีข้อมูล และความถี่น้ำท่วม)
# นิยาม: ถ้า freq มีค่า (ไม่เป็น NaN) = มีความเสี่ยง
def assess_risk(row):
    if pd.isna(row['freq']):
        return '0' #-ไม่เสี่ยง
    if row['freq'] >= 4: return '4' #-เสี่ยงสูงมาก
    if row['freq'] >= 3: return '3'#-เสี่ยงสูง
    if row['freq'] >= 2: return '2'#-เสี่ยงปานกลาง
    if row['freq'] >= 1: return '1'#-เสี่ยงน้อย

cleaned_farms['risk_level_new'] = cleaned_farms.apply(assess_risk, axis=1)

print("--- ตัวอย่างผลลัพธ์การจัดกลุ่มพืชและความเสี่ยง ---")
print(cleaned_farms[['act','type_name','detail_nam', 'crop_group', 'risk_level_new']].head(10))

--- ตัวอย่างผลลัพธ์การจัดกลุ่มพืชและความเสี่ยง ---
              act type_name detail_nam crop_group risk_level_new
226033  122650463      ข้าว   ข้าวเจ้า     1-ข้าว              2
59105   122746615      ข้าว   ข้าวเจ้า     1-ข้าว              0
87478   122840829      ข้าว   ข้าวเจ้า     1-ข้าว              0
139281  122879609      ข้าว   ข้าวเจ้า     1-ข้าว              0
13350   122881288      ข้าว   ข้าวเจ้า     1-ข้าว              0
72940   122881375      ข้าว   ข้าวเจ้า     1-ข้าว              0
55792   122925456      ข้าว   ข้าวเจ้า     1-ข้าว              0
125128  122925488      ข้าว   ข้าวเจ้า     1-ข้าว              0
226034  122997809      ข้าว   ข้าวเจ้า     1-ข้าว              0
226035  123024957      ข้าว   ข้าวเจ้า     1-ข้าว              0


In [ ]:
# 9. สรุปข้อมูลเป็นตาราง (แทนการใช้กราฟเพื่อเลี่ยงปัญหาฟอนต์ภาษาไทย)
print("--- ตารางสรุปจำนวนแปลงแยกตามระดับความเสี่ยง ---")
summary_table = cleaned_farms['risk_level_new'].value_counts().sort_index()
print(summary_table)



--- ตารางสรุปจำนวนแปลงแยกตามระดับความเสี่ยง ---
risk_level_new
0    316830
1     20226
2     16114
3      2548
4       156
Name: count, dtype: int64


In [ ]:
# 10. คัดเลือกเฉพาะ Field ที่ต้องการ (จัดระเบียบข้อมูล)
export_cols = [
    'p_name', 'a_name', 't_name', 'moo', 'act', 'year_act',
    'detail_nam', 'crop_group', 'plant_date', 'produce_da', 
    'freq', 'risk_level_new', 'geometry'
]
final_output = cleaned_farms[[c for c in export_cols if c in cleaned_farms.columns]].copy()

#  ส่งออกไฟล์
print("\n--- กำลังส่งออกไฟล์ ---")

# ส่งออกเป็น GeoPackage
final_output.to_file(r"E:\PythonFor_Flood\Flood_Analysis_Ayutthaya.gpkg", driver="GPKG")

# ส่งออก.csv 
final_output.drop(columns='geometry').to_csv(r"E:\PythonFor_Flood\Flood_Risk_Report_Ayutthaya.csv", index=False, encoding='utf-8-sig')

print("ส่งออกเป็นไฟล์ .csv เรียบร้อยแล้ว (สามารถใช้ Excel เปิดได้เลย)")


--- กำลังส่งออกไฟล์ ---
ส่งออกเป็นไฟล์ .csv เรียบร้อยแล้ว (สามารถใช้ Excel เปิดได้เลย)
